<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [147]:
!mkdir '/content/kamalo2018'
!mkdir '/content/kamalo2018/Images'
!mkdir '/content/kamalo2018/Labels'

!mkdir '/content/kamalo2019'
!mkdir '/content/kamalo2019/Images'
!mkdir '/content/kamalo2019/Labels'

!mkdir '/content/kamalo2020'
!mkdir '/content/kamalo2020/Images'
!mkdir '/content/kamalo2020/Labels'

In [148]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kamalo2018.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kakahaia2018.zip
   creating: /content/CCAP/content/kakahaia2018/Images/
  inflating: /content/CCAP/content/kakahaia2018/Images/1_1.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/1_250.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/1025_1.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/1025_250.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/1537_1.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/1537_250.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/2019_1.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/2019_250.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/513_1.tif  
  inflating: /content/CCAP/content/kakahaia2018/Images/513_250.tif  
   creating: /content/CCAP/content/kakahaia2018/Labels/
  inflating: /content/CCAP/content/kakahaia2018/Labels/1_1.tif  
  inflating: /content/CCAP/content/kakahaia2018

In [149]:
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kakahaia2019.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kakahaia2019.zip
   creating: /content/CCAP/content/kakahaia2019/Images/
  inflating: /content/CCAP/content/kakahaia2019/Images/1_1.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/1_250.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/1025_1.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/1025_250.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/1537_1.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/1537_250.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/2019_1.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/2019_250.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/513_1.tif  
  inflating: /content/CCAP/content/kakahaia2019/Images/513_250.tif  
   creating: /content/CCAP/content/kakahaia2019/Labels/
  inflating: /content/CCAP/content/kakahaia2019/Labels/1_1.tif  
  inflating: /content/CCAP/content/kakahaia2019

In [150]:
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kakahaia2020.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kakahaia2020.zip
   creating: /content/CCAP/content/kakahaia2020/Images/
  inflating: /content/CCAP/content/kakahaia2020/Images/1_1.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/1_250.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/1025_1.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/1025_250.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/1537_1.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/1537_250.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/2019_1.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/2019_250.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/513_1.tif  
  inflating: /content/CCAP/content/kakahaia2020/Images/513_250.tif  
   creating: /content/CCAP/content/kakahaia2020/Labels/
  inflating: /content/CCAP/content/kakahaia2020/Labels/1_1.tif  
  inflating: /content/CCAP/content/kakahaia2020

In [151]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kakahaia2018/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kakahaia2018/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 10
The number of files in the folder is: 10


In [152]:
df = pd.read_csv('/content/CCAP/content/kakahaia2018/kakahaia2018.csv')

In [153]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [154]:
# subset_df

In [155]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [156]:
# subset_df

In [157]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kakahaia2018/Images'
destination_images = '/content/kakahaia2018/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/kakahaia2018/Images
Copied 1_1.tif to /content/kakahaia2018/Images
Copied 1_1.tif to /content/kakahaia2018/Images
Copied 1_1.tif to /content/kakahaia2018/Images
Copied 1_1.tif to /content/kakahaia2018/Images
Copied 513_1.tif to /content/kakahaia2018/Images
Copied 513_1.tif to /content/kakahaia2018/Images
Copied 513_1.tif to /content/kakahaia2018/Images
Copied 513_1.tif to /content/kakahaia2018/Images
Copied 1025_1.tif to /content/kakahaia2018/Images
Copied 1025_1.tif to /content/kakahaia2018/Images
Copied 1537_1.tif to /content/kakahaia2018/Images
Copied 1537_1.tif to /content/kakahaia2018/Images
Copied 1_250.tif to /content/kakahaia2018/Images
Copied 1_250.tif to /content/kakahaia2018/Images
Copied 1_250.tif to /content/kakahaia2018/Images
Copied 1_250.tif to /content/kakahaia2018/Images
Copied 1_250.tif to /content/kakahaia2018/Images
Copied 513_250.tif to /content/kakahaia2018/Images
Copied 513_250.tif to /content/kakahaia2018/Images
Copied 513_250.tif to 

In [158]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kakahaia2018/Labels'
destination_labels = '/content/kakahaia2018/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/kakahaia2018/Labels
Copied 1_1.tif to /content/kakahaia2018/Labels
Copied 1_1.tif to /content/kakahaia2018/Labels
Copied 1_1.tif to /content/kakahaia2018/Labels
Copied 1_1.tif to /content/kakahaia2018/Labels
Copied 513_1.tif to /content/kakahaia2018/Labels
Copied 513_1.tif to /content/kakahaia2018/Labels
Copied 513_1.tif to /content/kakahaia2018/Labels
Copied 513_1.tif to /content/kakahaia2018/Labels
Copied 1025_1.tif to /content/kakahaia2018/Labels
Copied 1025_1.tif to /content/kakahaia2018/Labels
Copied 1537_1.tif to /content/kakahaia2018/Labels
Copied 1537_1.tif to /content/kakahaia2018/Labels
Copied 1_250.tif to /content/kakahaia2018/Labels
Copied 1_250.tif to /content/kakahaia2018/Labels
Copied 1_250.tif to /content/kakahaia2018/Labels
Copied 1_250.tif to /content/kakahaia2018/Labels
Copied 1_250.tif to /content/kakahaia2018/Labels
Copied 513_250.tif to /content/kakahaia2018/Labels
Copied 513_250.tif to /content/kakahaia2018/Labels
Copied 513_250.tif to 

In [159]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kakahaia2018/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kakahaia2018/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [160]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kakahaia2018_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kakahaia2018_subset.csv


In [161]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kakahaia2018'
zip_file_path = '/content/kakahaia2018_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kakahaia2018_subset.zip


In [162]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat


# 2019 tiles

In [163]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kakahaia2019/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kakahaia2019/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 10
The number of files in the folder is: 10


In [164]:
df = pd.read_csv('/content/CCAP/content/kakahaia2019/kakahaia2019.csv')

In [165]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [166]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [167]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kakahaia2019/Images'
destination_images = '/content/kakahaia2019/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/kakahaia2019/Images
Copied 1_1.tif to /content/kakahaia2019/Images
Copied 1_1.tif to /content/kakahaia2019/Images
Copied 1_1.tif to /content/kakahaia2019/Images
Copied 1_1.tif to /content/kakahaia2019/Images
Copied 513_1.tif to /content/kakahaia2019/Images
Copied 513_1.tif to /content/kakahaia2019/Images
Copied 513_1.tif to /content/kakahaia2019/Images
Copied 513_1.tif to /content/kakahaia2019/Images
Copied 1025_1.tif to /content/kakahaia2019/Images
Copied 1025_1.tif to /content/kakahaia2019/Images
Copied 1537_1.tif to /content/kakahaia2019/Images
Copied 1537_1.tif to /content/kakahaia2019/Images
Copied 1_250.tif to /content/kakahaia2019/Images
Copied 1_250.tif to /content/kakahaia2019/Images
Copied 1_250.tif to /content/kakahaia2019/Images
Copied 1_250.tif to /content/kakahaia2019/Images
Copied 1_250.tif to /content/kakahaia2019/Images
Copied 513_250.tif to /content/kakahaia2019/Images
Copied 513_250.tif to /content/kakahaia2019/Images
Copied 513_250.tif to 

In [168]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kakahaia2019/Labels'
destination_labels = '/content/kakahaia2019/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/kakahaia2019/Labels
Copied 1_1.tif to /content/kakahaia2019/Labels
Copied 1_1.tif to /content/kakahaia2019/Labels
Copied 1_1.tif to /content/kakahaia2019/Labels
Copied 1_1.tif to /content/kakahaia2019/Labels
Copied 513_1.tif to /content/kakahaia2019/Labels
Copied 513_1.tif to /content/kakahaia2019/Labels
Copied 513_1.tif to /content/kakahaia2019/Labels
Copied 513_1.tif to /content/kakahaia2019/Labels
Copied 1025_1.tif to /content/kakahaia2019/Labels
Copied 1025_1.tif to /content/kakahaia2019/Labels
Copied 1537_1.tif to /content/kakahaia2019/Labels
Copied 1537_1.tif to /content/kakahaia2019/Labels
Copied 1_250.tif to /content/kakahaia2019/Labels
Copied 1_250.tif to /content/kakahaia2019/Labels
Copied 1_250.tif to /content/kakahaia2019/Labels
Copied 1_250.tif to /content/kakahaia2019/Labels
Copied 1_250.tif to /content/kakahaia2019/Labels
Copied 513_250.tif to /content/kakahaia2019/Labels
Copied 513_250.tif to /content/kakahaia2019/Labels
Copied 513_250.tif to 

In [169]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kakahaia2019/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kakahaia2019/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [170]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kakahaia2019_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kakahaia2019_subset.csv


In [171]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kakahaia2019'
zip_file_path = '/content/kakahaia2019.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kakahaia2019.zip


In [172]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat


# 2020 tiles

In [173]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kakahaia2020/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kakahaia2020/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 10
The number of files in the folder is: 10


In [174]:
df = pd.read_csv('/content/CCAP/content/kakahaia2020/kakahaia2020.csv')

In [175]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [176]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [177]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kakahaia2020/Images'
destination_images = '/content/kakahaia2020/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1_1.tif to /content/kakahaia2020/Images
Copied 1_1.tif to /content/kakahaia2020/Images
Copied 1_1.tif to /content/kakahaia2020/Images
Copied 1_1.tif to /content/kakahaia2020/Images
Copied 1_1.tif to /content/kakahaia2020/Images
Copied 513_1.tif to /content/kakahaia2020/Images
Copied 513_1.tif to /content/kakahaia2020/Images
Copied 513_1.tif to /content/kakahaia2020/Images
Copied 513_1.tif to /content/kakahaia2020/Images
Copied 1025_1.tif to /content/kakahaia2020/Images
Copied 1025_1.tif to /content/kakahaia2020/Images
Copied 1537_1.tif to /content/kakahaia2020/Images
Copied 1537_1.tif to /content/kakahaia2020/Images
Copied 1_250.tif to /content/kakahaia2020/Images
Copied 1_250.tif to /content/kakahaia2020/Images
Copied 1_250.tif to /content/kakahaia2020/Images
Copied 1_250.tif to /content/kakahaia2020/Images
Copied 1_250.tif to /content/kakahaia2020/Images
Copied 513_250.tif to /content/kakahaia2020/Images
Copied 513_250.tif to /content/kakahaia2020/Images
Copied 513_250.tif to 

In [178]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kakahaia2020/Labels'
destination_labels = '/content/kakahaia2020/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1_1.tif to /content/kakahaia2020/Labels
Copied 1_1.tif to /content/kakahaia2020/Labels
Copied 1_1.tif to /content/kakahaia2020/Labels
Copied 1_1.tif to /content/kakahaia2020/Labels
Copied 1_1.tif to /content/kakahaia2020/Labels
Copied 513_1.tif to /content/kakahaia2020/Labels
Copied 513_1.tif to /content/kakahaia2020/Labels
Copied 513_1.tif to /content/kakahaia2020/Labels
Copied 513_1.tif to /content/kakahaia2020/Labels
Copied 1025_1.tif to /content/kakahaia2020/Labels
Copied 1025_1.tif to /content/kakahaia2020/Labels
Copied 1537_1.tif to /content/kakahaia2020/Labels
Copied 1537_1.tif to /content/kakahaia2020/Labels
Copied 1_250.tif to /content/kakahaia2020/Labels
Copied 1_250.tif to /content/kakahaia2020/Labels
Copied 1_250.tif to /content/kakahaia2020/Labels
Copied 1_250.tif to /content/kakahaia2020/Labels
Copied 1_250.tif to /content/kakahaia2020/Labels
Copied 513_250.tif to /content/kakahaia2020/Labels
Copied 513_250.tif to /content/kakahaia2020/Labels
Copied 513_250.tif to 

In [179]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kakahaia2020/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kakahaia2020/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 9
The number of files in the folder is: 9


In [180]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kakahaia2020_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kakahaia2020_subset.csv


In [181]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kakahaia2020'
zip_file_path = '/content/kakahaia2020_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kakahaia2020_subset.zip


In [182]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat


## Next 2018 tiles

In [183]:
!mkdir '/content/kaluaapuhi2018'
!mkdir '/content/kaluaapuhi2018/Images'
!mkdir '/content/kaluaapuhi2018/Labels'

!mkdir '/content/kaluaapuhi2019'
!mkdir '/content/kaluaapuhi2019/Images'
!mkdir '/content/kaluaapuhi2019/Labels'

!mkdir '/content/kaluaapuhi2020'
!mkdir '/content/kaluaapuhi2020/Images'
!mkdir '/content/kaluaapuhi2020/Labels'

In [184]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kaluaapuhi2018.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kaluaapuhi2018.zip
   creating: /content/CCAP/content/kaluaapuhi2018/Images/
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1_513.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1025_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1025_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1025_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1025_513.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1537_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1537_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1537_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2018/Images/1537_513.

In [185]:
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kaluaapuhi2019.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kaluaapuhi2019.zip
   creating: /content/CCAP/content/kaluaapuhi2019/Images/
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1_513.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1025_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1025_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1025_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1025_513.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1537_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1537_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1537_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2019/Images/1537_513.

In [186]:
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kaluaapuhi2020.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kaluaapuhi2020.zip
   creating: /content/CCAP/content/kaluaapuhi2020/Images/
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1_513.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1025_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1025_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1025_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1025_513.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1537_1.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1537_1025.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1537_1057.tif  
  inflating: /content/CCAP/content/kaluaapuhi2020/Images/1537_513.

In [187]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kaluaapuhi2018/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kaluaapuhi2018/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 32
The number of files in the folder is: 32


In [188]:
df = pd.read_csv('/content/CCAP/content/kaluaapuhi2018/kaluaapuhi2018.csv')

In [189]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [190]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [191]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kaluaapuhi2018/Images'
destination_images = '/content/kaluaapuhi2018/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1025_1.tif to /content/kaluaapuhi2018/Images
Copied 1537_1.tif to /content/kaluaapuhi2018/Images
Copied 1537_1.tif to /content/kaluaapuhi2018/Images
Copied 1537_1.tif to /content/kaluaapuhi2018/Images
Copied 1537_1.tif to /content/kaluaapuhi2018/Images
Copied 1_513.tif to /content/kaluaapuhi2018/Images
Copied 1_513.tif to /content/kaluaapuhi2018/Images
Copied 513_513.tif to /content/kaluaapuhi2018/Images
Copied 513_513.tif to /content/kaluaapuhi2018/Images
Copied 513_513.tif to /content/kaluaapuhi2018/Images
Copied 513_513.tif to /content/kaluaapuhi2018/Images
Copied 1025_513.tif to /content/kaluaapuhi2018/Images
Copied 1025_513.tif to /content/kaluaapuhi2018/Images
Copied 1025_513.tif to /content/kaluaapuhi2018/Images
Copied 1025_513.tif to /content/kaluaapuhi2018/Images
Copied 1537_513.tif to /content/kaluaapuhi2018/Images
Copied 1537_513.tif to /content/kaluaapuhi2018/Images
Copied 1537_513.tif to /content/kaluaapuhi2018/Images
Copied 2049_513.tif to /content/kaluaapuhi2018/I

In [192]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kaluaapuhi2018/Labels'
destination_labels = '/content/kaluaapuhi2018/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1025_1.tif to /content/kaluaapuhi2018/Labels
Copied 1537_1.tif to /content/kaluaapuhi2018/Labels
Copied 1537_1.tif to /content/kaluaapuhi2018/Labels
Copied 1537_1.tif to /content/kaluaapuhi2018/Labels
Copied 1537_1.tif to /content/kaluaapuhi2018/Labels
Copied 1_513.tif to /content/kaluaapuhi2018/Labels
Copied 1_513.tif to /content/kaluaapuhi2018/Labels
Copied 513_513.tif to /content/kaluaapuhi2018/Labels
Copied 513_513.tif to /content/kaluaapuhi2018/Labels
Copied 513_513.tif to /content/kaluaapuhi2018/Labels
Copied 513_513.tif to /content/kaluaapuhi2018/Labels
Copied 1025_513.tif to /content/kaluaapuhi2018/Labels
Copied 1025_513.tif to /content/kaluaapuhi2018/Labels
Copied 1025_513.tif to /content/kaluaapuhi2018/Labels
Copied 1025_513.tif to /content/kaluaapuhi2018/Labels
Copied 1537_513.tif to /content/kaluaapuhi2018/Labels
Copied 1537_513.tif to /content/kaluaapuhi2018/Labels
Copied 1537_513.tif to /content/kaluaapuhi2018/Labels
Copied 2049_513.tif to /content/kaluaapuhi2018/L

In [193]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kaluaapuhi2018/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kaluaapuhi2018/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 23
The number of files in the folder is: 23


In [194]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kaluaapuhi2018_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kaluaapuhi2018_subset.csv


In [195]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kaluaapuhi2018'
zip_file_path = '/content/kaluaapuhi2018_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kaluaapuhi2018_subset.zip


In [196]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat


## next 2019 tiles

In [217]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kaluaapuhi2019/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kaluaapuhi2019/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 32
The number of files in the folder is: 32


In [218]:
df = pd.read_csv('/content/CCAP/content/kaluaapuhi2019/kaluaapuhi2019.csv')

In [219]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [220]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [221]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kaluaapuhi2019/Images'
destination_images = '/content/kaluaapuhi2019/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1025_1.tif to /content/kaluaapuhi2019/Images
Copied 1537_1.tif to /content/kaluaapuhi2019/Images
Copied 1537_1.tif to /content/kaluaapuhi2019/Images
Copied 1537_1.tif to /content/kaluaapuhi2019/Images
Copied 1537_1.tif to /content/kaluaapuhi2019/Images
Copied 1_513.tif to /content/kaluaapuhi2019/Images
Copied 1_513.tif to /content/kaluaapuhi2019/Images
Copied 513_513.tif to /content/kaluaapuhi2019/Images
Copied 513_513.tif to /content/kaluaapuhi2019/Images
Copied 513_513.tif to /content/kaluaapuhi2019/Images
Copied 513_513.tif to /content/kaluaapuhi2019/Images
Copied 1025_513.tif to /content/kaluaapuhi2019/Images
Copied 1025_513.tif to /content/kaluaapuhi2019/Images
Copied 1025_513.tif to /content/kaluaapuhi2019/Images
Copied 1025_513.tif to /content/kaluaapuhi2019/Images
Copied 1537_513.tif to /content/kaluaapuhi2019/Images
Copied 1537_513.tif to /content/kaluaapuhi2019/Images
Copied 1537_513.tif to /content/kaluaapuhi2019/Images
Copied 2049_513.tif to /content/kaluaapuhi2019/I

In [222]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kaluaapuhi2019/Labels'
destination_labels = '/content/kaluaapuhi2019/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1025_1.tif to /content/kaluaapuhi2019/Labels
Copied 1537_1.tif to /content/kaluaapuhi2019/Labels
Copied 1537_1.tif to /content/kaluaapuhi2019/Labels
Copied 1537_1.tif to /content/kaluaapuhi2019/Labels
Copied 1537_1.tif to /content/kaluaapuhi2019/Labels
Copied 1_513.tif to /content/kaluaapuhi2019/Labels
Copied 1_513.tif to /content/kaluaapuhi2019/Labels
Copied 513_513.tif to /content/kaluaapuhi2019/Labels
Copied 513_513.tif to /content/kaluaapuhi2019/Labels
Copied 513_513.tif to /content/kaluaapuhi2019/Labels
Copied 513_513.tif to /content/kaluaapuhi2019/Labels
Copied 1025_513.tif to /content/kaluaapuhi2019/Labels
Copied 1025_513.tif to /content/kaluaapuhi2019/Labels
Copied 1025_513.tif to /content/kaluaapuhi2019/Labels
Copied 1025_513.tif to /content/kaluaapuhi2019/Labels
Copied 1537_513.tif to /content/kaluaapuhi2019/Labels
Copied 1537_513.tif to /content/kaluaapuhi2019/Labels
Copied 1537_513.tif to /content/kaluaapuhi2019/Labels
Copied 2049_513.tif to /content/kaluaapuhi2019/L

In [281]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kaluaapuhi2019/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kaluaapuhi2019/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 23
The number of files in the folder is: 23


In [282]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kaluaapuhi2019_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kaluaapuhi2019_subset.csv


In [283]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kaluaapuhi2019'
zip_file_path = '/content/kaluaapuhi2019_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kaluaapuhi2019_subset.zip


In [284]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat


## next 2020 tiles

In [285]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kaluaapuhi2020/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kaluaapuhi2020/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 32
The number of files in the folder is: 32


In [286]:
df = pd.read_csv('/content/CCAP/content/kaluaapuhi2020/kaluaapuhi2020.csv')

In [287]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [288]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [289]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kaluaapuhi2020/Images'
destination_images = '/content/kaluaapuhi2020/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1025_1.tif to /content/kaluaapuhi2020/Images
Copied 1537_1.tif to /content/kaluaapuhi2020/Images
Copied 1537_1.tif to /content/kaluaapuhi2020/Images
Copied 1537_1.tif to /content/kaluaapuhi2020/Images
Copied 1537_1.tif to /content/kaluaapuhi2020/Images
Copied 1_513.tif to /content/kaluaapuhi2020/Images
Copied 1_513.tif to /content/kaluaapuhi2020/Images
Copied 513_513.tif to /content/kaluaapuhi2020/Images
Copied 513_513.tif to /content/kaluaapuhi2020/Images
Copied 513_513.tif to /content/kaluaapuhi2020/Images
Copied 513_513.tif to /content/kaluaapuhi2020/Images
Copied 1025_513.tif to /content/kaluaapuhi2020/Images
Copied 1025_513.tif to /content/kaluaapuhi2020/Images
Copied 1025_513.tif to /content/kaluaapuhi2020/Images
Copied 1025_513.tif to /content/kaluaapuhi2020/Images
Copied 1537_513.tif to /content/kaluaapuhi2020/Images
Copied 1537_513.tif to /content/kaluaapuhi2020/Images
Copied 1537_513.tif to /content/kaluaapuhi2020/Images
Copied 2049_513.tif to /content/kaluaapuhi2020/I

In [290]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kaluaapuhi2020/Labels'
destination_labels = '/content/kaluaapuhi2020/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1025_1.tif to /content/kaluaapuhi2020/Labels
Copied 1537_1.tif to /content/kaluaapuhi2020/Labels
Copied 1537_1.tif to /content/kaluaapuhi2020/Labels
Copied 1537_1.tif to /content/kaluaapuhi2020/Labels
Copied 1537_1.tif to /content/kaluaapuhi2020/Labels
Copied 1_513.tif to /content/kaluaapuhi2020/Labels
Copied 1_513.tif to /content/kaluaapuhi2020/Labels
Copied 513_513.tif to /content/kaluaapuhi2020/Labels
Copied 513_513.tif to /content/kaluaapuhi2020/Labels
Copied 513_513.tif to /content/kaluaapuhi2020/Labels
Copied 513_513.tif to /content/kaluaapuhi2020/Labels
Copied 1025_513.tif to /content/kaluaapuhi2020/Labels
Copied 1025_513.tif to /content/kaluaapuhi2020/Labels
Copied 1025_513.tif to /content/kaluaapuhi2020/Labels
Copied 1025_513.tif to /content/kaluaapuhi2020/Labels
Copied 1537_513.tif to /content/kaluaapuhi2020/Labels
Copied 1537_513.tif to /content/kaluaapuhi2020/Labels
Copied 1537_513.tif to /content/kaluaapuhi2020/Labels
Copied 2049_513.tif to /content/kaluaapuhi2020/L

In [291]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kaluaapuhi2020/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kaluaapuhi2020/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 23
The number of files in the folder is: 23


In [292]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kaluaapuhi2020_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kaluaapuhi2020_subset.csv


In [293]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kaluaapuhi2020'
zip_file_path = '/content/kaluaapuhi2020_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/kaluaapuhi2020_subset.zip


In [294]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata/landsat


## 2018 tiles

In [295]:
!mkdir '/content/kahuku2018'
!mkdir '/content/kahuku2018/Images'
!mkdir '/content/kahuku2018/Labels'

!mkdir '/content/kahuku2019'
!mkdir '/content/kahuku2019/Images'
!mkdir '/content/kahuku2019/Labels'

!mkdir '/content/kahuku2020'
!mkdir '/content/kahuku2020/Images'
!mkdir '/content/kahuku2020/Labels'

mkdir: cannot create directory ‘/content/kahuku2018’: File exists
mkdir: cannot create directory ‘/content/kahuku2018/Images’: File exists
mkdir: cannot create directory ‘/content/kahuku2018/Labels’: File exists
mkdir: cannot create directory ‘/content/kahuku2019’: File exists
mkdir: cannot create directory ‘/content/kahuku2019/Images’: File exists
mkdir: cannot create directory ‘/content/kahuku2019/Labels’: File exists
mkdir: cannot create directory ‘/content/kahuku2020’: File exists
mkdir: cannot create directory ‘/content/kahuku2020/Images’: File exists
mkdir: cannot create directory ‘/content/kahuku2020/Labels’: File exists


In [296]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kahuku2018.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kahuku2018.zip
replace /content/CCAP/content/kahuku2018/Images/1_1.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [297]:
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kahuku2019.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kahuku2019.zip
replace /content/CCAP/content/kahuku2019/Images/1_1.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [298]:
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kahuku2020.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/landsat/kahuku2020.zip
replace /content/CCAP/content/kahuku2020/Images/1_1.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [299]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2018/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2018/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 24
The number of files in the folder is: 24


In [300]:
df = pd.read_csv('/content/CCAP/content/kahuku2018/kahuku2018.csv')

In [301]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [302]:
# subset_df

In [303]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [304]:
# subset_df

In [305]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2018/Images'
destination_images = '/content/kahuku2018/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1537_1.tif to /content/kahuku2018/Images
Copied 513_513.tif to /content/kahuku2018/Images
Copied 513_513.tif to /content/kahuku2018/Images
Copied 1025_513.tif to /content/kahuku2018/Images
Copied 1025_513.tif to /content/kahuku2018/Images
Copied 1025_513.tif to /content/kahuku2018/Images
Copied 1537_513.tif to /content/kahuku2018/Images
Copied 1537_513.tif to /content/kahuku2018/Images
Copied 2049_513.tif to /content/kahuku2018/Images
Copied 1025_1025.tif to /content/kahuku2018/Images
Copied 1537_1025.tif to /content/kahuku2018/Images
Copied 1537_1025.tif to /content/kahuku2018/Images
Copied 1537_1025.tif to /content/kahuku2018/Images
Copied 2049_1025.tif to /content/kahuku2018/Images
Copied 2049_1025.tif to /content/kahuku2018/Images
Copied 2274_1025.tif to /content/kahuku2018/Images
Copied 2274_1025.tif to /content/kahuku2018/Images
Copied 1537_1388.tif to /content/kahuku2018/Images
Copied 1537_1388.tif to /content/kahuku2018/Images
Copied 2049_1388.tif to /content/kahuku2018/

In [306]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2018/Labels'
destination_labels = '/content/kahuku2018/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1537_1.tif to /content/kahuku2018/Labels
Copied 513_513.tif to /content/kahuku2018/Labels
Copied 513_513.tif to /content/kahuku2018/Labels
Copied 1025_513.tif to /content/kahuku2018/Labels
Copied 1025_513.tif to /content/kahuku2018/Labels
Copied 1025_513.tif to /content/kahuku2018/Labels
Copied 1537_513.tif to /content/kahuku2018/Labels
Copied 1537_513.tif to /content/kahuku2018/Labels
Copied 2049_513.tif to /content/kahuku2018/Labels
Copied 1025_1025.tif to /content/kahuku2018/Labels
Copied 1537_1025.tif to /content/kahuku2018/Labels
Copied 1537_1025.tif to /content/kahuku2018/Labels
Copied 1537_1025.tif to /content/kahuku2018/Labels
Copied 2049_1025.tif to /content/kahuku2018/Labels
Copied 2049_1025.tif to /content/kahuku2018/Labels
Copied 2274_1025.tif to /content/kahuku2018/Labels
Copied 2274_1025.tif to /content/kahuku2018/Labels
Copied 1537_1388.tif to /content/kahuku2018/Labels
Copied 1537_1388.tif to /content/kahuku2018/Labels
Copied 2049_1388.tif to /content/kahuku2018/

In [307]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2018/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2018/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 12
The number of files in the folder is: 12


In [308]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2018_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/kahuku2018_subset.csv


In [309]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2018'
zip_file_path = '/content/kahuku2018_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

KeyboardInterrupt: 

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

## 2019 tiles

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2019/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2019/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/kahuku2019/kahuku2019.csv')

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
# subset_df

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
# subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2019/Images'
destination_images = '/content/kahuku2019/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2019/Labels'
destination_labels = '/content/kahuku2019/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2019/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2019/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2019_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2019'
zip_file_path = '/content/kahuku2019_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

## 2020 cells

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2020/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2020/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/kahuku2020/kahuku2020.csv')

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
# subset_df

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
# subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2020/Images'
destination_images = '/content/kahuku2020/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2020/Labels'
destination_labels = '/content/kahuku2020/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2020/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2020/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2020_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2020'
zip_file_path = '/content/kahuku2020_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata/landsat'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')